In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from matplotlib import style
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import os

style.use('ggplot')

POS_TRAIN_PATH = '../datasets/aclImdb/train/pos/'
NEG_TRAIN_PATH = '../datasets//aclImdb/train/neg/'
pos_files = os.listdir(POS_TRAIN_PATH)
neg_files = os.listdir(NEG_TRAIN_PATH)
print('Docs prep....')
pos_lines = []
neg_lines = []
for file in pos_files:
    with open(POS_TRAIN_PATH + file) as f:
        lines = f.readlines()
        lines = [line.strip() for line in lines if line.strip()]
        pos_lines.extend(lines)
for file in neg_files:
    with open(NEG_TRAIN_PATH + file) as f:
        lines = f.readlines()
        lines = [line.strip() for line in lines if line.strip()]
        neg_lines.extend(lines)

docs = []
docs.extend(pos_lines)
docs.extend(neg_lines)
print('Vectorize...')
vectorizer2 = TfidfVectorizer(stop_words=ENGLISH_STOP_WORDS, max_features=200)
vectorizer2.fit(docs)
print('Vectorization done')

input_data2 = vectorizer2.transform(docs).todense()
print(input_data2.shape)
df2 = pd.DataFrame(input_data2)

pos_labels = np.zeros(len(pos_lines))
neg_labels = np.ones(len(neg_lines))

labels2 = []
labels2 = np.append(pos_labels, neg_labels)
df2['labels'] = labels2
# df2.to_csv('data/aclImdb/processed_input_labelled3.csv')

pos_labels = np.zeros(len(pos_lines))
neg_labels = np.ones(len(neg_lines))

X = df2.drop('labels', axis=1)
X = np.array(X)
print(X.shape)

X_train, X_test, y_train, y_test = train_test_split(X, df2['labels'])


Docs prep....
Vectorize...
Vectorization done
(25000, 200)
(25000, 200)


In [35]:
y_train.dtype


dtype('float64')

## By Extending Dataset

In [1]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
class TextLoader(Dataset):
    def __init__(self,X_train, y_train):
        self.X_train =  np.array(X_train)
        self.y_train = np.array(y_train)
   
    def __getitem__(self, index):
        return self.X_train[index], self.y_train[index]
    
    def __len__(self):
        
        return len(self.X_train.tolist())


In [4]:
dset = TextLoader(X_train, y_train)  

trainLoader = DataLoader(dset, batch_size=100, shuffle=True, num_workers=4)

In [6]:
x, y = list(trainLoader)[0]
print(x.size(), y.size())

torch.Size([100, 200]) torch.Size([100])


## Secondary Method

In [7]:
import torch.utils.data as data_utils
import numpy as np

In [8]:
train = data_utils.TensorDataset(torch.from_numpy(np.array(X_train)), torch.from_numpy(np.array(y_train)))

In [9]:
train_loader = data_utils.DataLoader(train, batch_size=50, shuffle=True)

In [10]:
x, y = list(train_loader)[0]
print(x.size(), y.size())

torch.Size([50, 200]) torch.Size([50])
